In [24]:
import pandas as pd
import numpy as np

In [25]:
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [26]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [27]:
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

# TO-DO:
1) 모델 학습하고
2) X_test 예측하고
3) sample_submission.csv 만들어서 제출

sample_submission에다가 prediction을 덮고 .csv 파일로 쓰기

In [28]:
# 쓸 변수만 골라서 새로운 데이터 만들기
features = ['LotArea', 'Utilities', 'GrLivArea']

# 훈련용 입력값(땅 넓이, 설비, 지상층 거실 크기)
X_train = df_train[features].copy() # 학습에 쓸 데이터 값만 골라서 저장. 원본X

# 훈련용 정답값(집값)
y_train = df_train['SalePrice'] # 맞춰야 하는 값

# 테스트용 입력값
X_test = df_test[features].copy()

In [29]:
# 쓸 데이터 자료 확인하기
print(X_train['LotArea'].dtypes)
print(X_train['Utilities'].dtypes)
print(X_train['GrLivArea'].dtypes)

int64
object
int64


In [30]:
# Object 떴는데 어떻게 생겼을까
print(X_train['Utilities'].head(10))

0    AllPub
1    AllPub
2    AllPub
3    AllPub
4    AllPub
5    AllPub
6    AllPub
7    AllPub
8    AllPub
9    AllPub
Name: Utilities, dtype: object


In [31]:
# 글자 → 숫자 (간단한 방법: Label Encoding)
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# X_train['Utilities'] = le.fit_transform(X_train['Utilities'].astype(str))
# X_test['Utilities'] = le.transform(X_test['Utilities'].astype(str))

# 했는데 오류남..

In [32]:
# 두 데이터셋이 완전히 같은 구조라고 절대 단정짓지 말기.. 확인해야기
print(X_train['Utilities'].unique())
print(X_test['Utilities'].unique())

['AllPub' 'NoSeWa']
['AllPub' nan]


In [33]:
from sklearn.preprocessing import LabelEncoder

# Nan 없애기 -> 학습 시킬 수가 없음
X_train['Utilities'] = X_train['Utilities'].fillna('None')
X_test['Utilities'] = X_test['Utilities'].fillna('None')

# 전부 문자열로 바꾸기 (LabelEncoder가 한가지 타입으로만 읽음)
combined_utilities = pd.concat([
    X_train['Utilities'].astype(str),
    X_test['Utilities'].astype(str)
])

# Label Encoding
le = LabelEncoder()
le.fit(combined_utilities)

X_train['Utilities'] = le.transform(X_train['Utilities'].astype(str))
X_test['Utilities'] = le.transform(X_test['Utilities'].astype(str))

In [34]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [35]:
preds = model.predict(X_test)

In [36]:
submission = pd.DataFrame({
    'Id': df_test['Id'],
    'SalePrice': preds
})

In [37]:
submission.to_csv("submission.csv", index=False)